# Тема 7
### Вариант 4
$0=2\ln x -1/x$ \
все корни

In [15]:
import signal
from functools import wraps
import time
import numpy as np
from pprint import pprint
import sympy as sp
from tabulate import tabulate

In [16]:
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

<function __main__.timeout_handler(signum, frame)>

In [17]:
def count_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.monotonic()
        try:
            signal.alarm(10)
            res = func(*args, **kwargs)
        except TimeoutException:
            print("таймаут 10 секунд")
            return None
        finally:
            signal.alarm(0)
        
        result_time = time.monotonic() - start_time
        print(f"{result_time=:6f}")
        return res
    return wrapper

In [18]:
ACCURACY_ORDER = 10
ACCURACY = 1 / (10**ACCURACY_ORDER)

In [19]:
x_sym = sp.Symbol('x')
f_sym = 2 * sp.ln(x_sym) - 1 / x_sym
f_sym = sp.simplify(f_sym)
f_prime_sym = sp.diff(f_sym, x_sym)
f_prime_sym = sp.simplify(f_prime_sym)

f = sp.lambdify(x_sym, f_sym, 'numpy')
f_prime = sp.lambdify(x_sym, f_prime_sym, 'numpy')


pprint(f_sym)
pprint(f_prime_sym)

2*log(x) - 1/x
(2*x + 1)/x**2


In [26]:
# находим нули производной
critical_points = sp.solve(f_prime_sym, x_sym)
# critical_points = [np.float128(i) for i in critical_points]
pprint(critical_points)


[-1/2]


In [21]:
# разбиваем интервалы нулями производной
intervals = [[np.float128(0.00000001), np.float128(5)]]
for critical_point in critical_points:
    stop_i = len(intervals)
    i = 0
    while i < stop_i:
        if intervals[i][0] < critical_point < intervals[i][1]:
            st, en = intervals[i]
            intervals[i][1] = critical_point
            intervals.append([critical_point, en])
            intervals.sort()
            stop_i += 1
        i += 1
intervals


[[np.longdouble('1.0000000000000000209e-08'), np.longdouble('5.0')]]

In [22]:
@count_time
def solve_by_method(
    a: float,
    b: float,
    accuracy: float = ACCURACY,
    with_logging: bool = True
) -> np.float128:
    steps = [[a, b]]

    while abs(a-b) > accuracy:
        # метод касательных
        a_new = a - f(a)/f_prime(a)

        # метод хорд
        b_new = b - (f(b)*(a-b)) / (f(a)-f(b))

        if with_logging:
            steps.append([float(a_new), float(b_new)])

        a, b = a_new, b_new
    result = float((a+b) / 2)
    if with_logging:
        print(f"f({result}) = 0")
    if f(result) > accuracy:
        raise ArithmeticError(f"не смогло хорошо посчитать {result=} {f(result)=}")
    if with_logging:
        print(
            tabulate(
                steps,
                headers=["x̄(a)", "x(b)"],
                tablefmt="grid",
            )
        )

    if with_logging:
        print(f"f({result}) = 0")
    return result




In [25]:
# results = []
# for interval in intervals:
#     results.append(solve_by_method(*interval))
solve_by_method(0.000001, 10)

f(1.4215299358831168) = 0
+-------------+----------+
|        x̄(a) |     x(b) |
+=============+==========+
| 1e-06       | 10       |
+-------------+----------+
| 2.00003e-06 |  9.99995 |
+-------------+----------+
| 4.00015e-06 |  9.99986 |
+-------------+----------+
| 8.00066e-06 |  9.99968 |
+-------------+----------+
| 1.60027e-05 |  9.99932 |
+-------------+----------+
| 3.20105e-05 |  9.9986  |
+-------------+----------+
| 6.40402e-05 |  9.99716 |
+-------------+----------+
| 0.000128151 |  9.99428 |
+-------------+----------+
| 0.000256564 |  9.98853 |
+-------------+----------+
| 0.000514085 |  9.97705 |
+-------------+----------+
| 0.00103164  |  9.9542  |
+-------------+----------+
| 0.00207577  |  9.9089  |
+-------------+----------+
| 0.00419596  |  9.81976 |
+-------------+----------+
| 0.00854814  |  9.64696 |
+-------------+----------+
| 0.0176368   |  9.3209  |
+-------------+----------+
| 0.0370991   |  8.73454 |
+-------------+----------+
| 0.0800771   |  7.76259 |
+

1.4215299358831168

In [ ]:
print("результаты")
print(f"f(x)={f_sym}")
print(f"разбили на интервалы {intervals}")
print(f"решения на интервалах {results}")

результаты
f(x)=2*log(x) - 1/x
разбили на интервалы [[np.longdouble('1.0000000000000000209e-08'), np.longdouble('5.0')]]
решения на интервалах [1.4215299358831166]


# мой интерес на сравнить этот метод с подобием бин поиска

In [ ]:
# pos = is_positive
@count_time
def solve_by_binary_search(
    a: float,
    b: float,
    accuracy: float = ACCURACY,
    with_logging: bool = True
) -> np.float128:
    a, b = interval
    a_pos = f(a) >= 0
    b_pos = f(b) >= 0
    steps = [[a, b]]

    while abs(a-b) > accuracy:
        mid = (b+a) / 2
        mid_pos = f(mid) >= 0
        if mid_pos == a_pos:
            # ответ в [mid, b]
            a = mid
            a_pos = mid_pos
        elif mid_pos == b_pos:
            # ответ в [a, mid]
            b = mid
            b_pos = mid_pos
        else:
            raise ArithmeticError(f"слева и справа - один знак {a=} {b=}")

        if with_logging:
            steps.append([float(a), float(b)])
    
    result = float((a+b) / 2)
    if with_logging:
        print(f"f({result}) = 0")
    if f(result) > accuracy:
        raise ArithmeticError(f"не смогло хорошо посчитать {result=} {f(result)=}")

    if with_logging:
        print(
            tabulate(
                steps,
                headers=["x̄(a)", "x(b)"],
                tablefmt="grid",
            )
        )

    if with_logging:
        print(f"f({result}) = 0")
    return result



In [ ]:
for interval in intervals:
    solve_by_binary_search(*interval)

f(1.4215299358958318) = 0
+---------+---------+
|    x̄(a) |    x(b) |
+=========+=========+
| 1e-08   | 5       |
+---------+---------+
| 1e-08   | 2.5     |
+---------+---------+
| 1.25    | 2.5     |
+---------+---------+
| 1.25    | 1.875   |
+---------+---------+
| 1.25    | 1.5625  |
+---------+---------+
| 1.40625 | 1.5625  |
+---------+---------+
| 1.40625 | 1.48438 |
+---------+---------+
| 1.40625 | 1.44531 |
+---------+---------+
| 1.40625 | 1.42578 |
+---------+---------+
| 1.41602 | 1.42578 |
+---------+---------+
| 1.4209  | 1.42578 |
+---------+---------+
| 1.4209  | 1.42334 |
+---------+---------+
| 1.4209  | 1.42212 |
+---------+---------+
| 1.42151 | 1.42212 |
+---------+---------+
| 1.42151 | 1.42181 |
+---------+---------+
| 1.42151 | 1.42166 |
+---------+---------+
| 1.42151 | 1.42159 |
+---------+---------+
| 1.42151 | 1.42155 |
+---------+---------+
| 1.42153 | 1.42155 |
+---------+---------+
| 1.42153 | 1.42154 |
+---------+---------+
| 1.42153 | 1.42153 |
+----

### само сравнение

In [ ]:
for order in range(1, 22):
    accuracy = 1/(10**order)
    print(f"считаем для порядка {order} {accuracy=}")
    for interval in intervals:
        print("метод хорд и касательных", end=": ")
        solve_by_method(*interval, accuracy=accuracy, with_logging=False)
        print("бинпоиск", end=": ")
        solve_by_binary_search(*interval, accuracy=accuracy, with_logging=False)
        print("---")

считаем для порядка 1 accuracy=0.1
метод хорд и касательных: result_time=0.000163
бинпоиск: result_time=0.000025
---
считаем для порядка 2 accuracy=0.01
метод хорд и касательных: result_time=0.000086
бинпоиск: result_time=0.000020
---
считаем для порядка 3 accuracy=0.001
метод хорд и касательных: result_time=0.000132
бинпоиск: result_time=0.000049
---
считаем для порядка 4 accuracy=0.0001
метод хорд и касательных: result_time=0.000149
бинпоиск: result_time=0.000063
---
считаем для порядка 5 accuracy=1e-05
метод хорд и касательных: result_time=0.000147
бинпоиск: result_time=0.000065
---
считаем для порядка 6 accuracy=1e-06
метод хорд и касательных: result_time=0.000139
бинпоиск: result_time=0.000070
---
считаем для порядка 7 accuracy=1e-07
метод хорд и касательных: result_time=0.000174
бинпоиск: result_time=0.000080
---
считаем для порядка 8 accuracy=1e-08
метод хорд и касательных: result_time=0.000148
бинпоиск: result_time=0.000082
---
считаем для порядка 9 accuracy=1e-09
метод хорд и 

### вывод
на маленком порядке точности бинпоиск работает быстрее \
но с увеличением порядка бинпоиск >18 бинпоиск становится максимально неэффективным